# Reconhecimento de Padrões (TIP8311) - Trabalho 2


**Professor:** Guilherme de Alencar Barreto  

<img src="https://loop.frontiersin.org/images/profile/243428/203" alt="Foto do Professor" width="150"/>


**Aluno:** Luis Felipe Carneiro de Souza    **Matrícula:** 593034

In [3]:
import numpy as np
from tqdm import tqdm
from time import time
import matplotlib.pyplot as plt

In [32]:
data_path = "vertebral+column\column_3C.dat"

In [33]:
data = np.genfromtxt(fname=data_path, delimiter=None, dtype=str, encoding='utf-8')
data

X = data[:, :-1].astype(float) 
y = data[:, -1]


In [18]:
class KNN:
    def __init__(self, k=1, m=2):
        self.X_train = None
        self.y_train = None
        self.k = k
        self.m = m

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):

        n_test = len(X)
        n_train = len(self.X_train)
        y_pred = []

        for i in range(n_test):

            dist = []

            for j in range(n_train):

                diff = np.abs(X[i, :] - self.X_train[j , :])
                dist.append(np.sum(diff ** (self.m)))

            dist = np.array(dist)
            idx = np.argsort(dist)[:self.k]
            neighbors, count = np.unique(self.y_train[idx], return_counts=True)
            y_pred.append(neighbors[np.argmax(count)])

        return np.array(y_pred).reshape(-1, 1)
        

In [39]:
class DMC:
    def __init__(self):
        self.centroides = None
        self.classes = None

    def fit(self, X, y):

        self.classes = np.sort(np.unique(y))
        self.centroides = []

        for c in self.classes:

            mask = (y == c)
            self.centroides.append(X[mask].mean(axis=0))

        self.centroides = np.array(self.centroides)

    def predict(self, X):

        n_test = len(X)
        y_pred = []

        for i in range(n_test):

            dist = []

            for c in self.centroides:

                diff = np.abs(c - X[i, :])
                dist.append(np.sum(diff ** 2))

            dist = np.array(dist)
            y_pred.append(self.classes[np.argmin(dist)])

        return np.array(y_pred).ravel()

In [20]:
class DMCR:
    def __init__(self):
        self.centroides = None
        self.classes = None

    def fit(self, X, y):

        self.classes = np.sort(np.unique(y))
        self.centroides = []

        for c in self.classes:

            mask = (y == c)
            self.centroides.append(np.median(X[mask], axis=0))

        self.centroides = np.array(self.centroides)

    def predict(self, X):

        n_test = len(X)
        y_pred = []

        for i in range(n_test):

            dist = []

            for c in self.centroides:

                diff = np.abs(c - X[i, :])
                dist.append(np.sum(diff))

            dist = np.array(dist)
            y_pred.append(self.classes[np.argmin(dist)])

        return np.array(y_pred).reshape(-1, 1)

In [21]:
class MaxCorr:
    def __init__(self):
        self.centroides = None
        self.classes = None

    def fit(self, X, y):

        self.classes = np.sort(np.unique(y))
        self.centroides = []

        for c in self.classes:

            mask = (y == c)
            self.centroides.append(X[mask].mean(axis=0))

        self.centroides = np.array(self.centroides)

    def predict(self, X):

        n_test = len(X)
        y_pred = []

        for i in range(n_test):

            corr_array = []

            for c in self.centroides:

                corr = np.dot(c, X[i, :])
                corr_array.append(corr)

            corr_array = np.array(corr_array)
            y_pred.append(self.classes[np.argmax(corr_array)])

        return np.array(y_pred).reshape(-1, 1)

In [22]:
class StandardScaler:
    def __init__(self):
        self.X_mu = None
        self.X_sigma = None
        self.y_mu = None
        self.y_sigma = None

    def fit(self, X, y=None):
        self.X_mu = np.mean(X, axis=0)
        self.X_sigma = np.std(X, axis=0)
        self.X_sigma = np.where(self.X_sigma == 0, 1.0, self.X_sigma)

        if y is not None:
            self.y_mu = np.mean(y, axis=0)
            self.y_sigma = np.std(y, axis=0)

    def transform(self, X, y=None):
        
        X_scaled = (X - self.X_mu) / self.X_sigma

        if y is not None:
            y_scaled = (y - self.y_mu) / self.y_sigma

        if y is None:
            return X_scaled
        else:
            return X_scaled, y_scaled

In [ ]:
class MinMaxScaler:
    def __init__(self):
        pass

    def fit(self, X, y=None):
        pass

    def predict(self, X, y=None):
        pass

In [35]:
def accuracy_score(y_true, y_pred):
    return float(np.mean(y_true == y_pred))

In [49]:
def confusion_matrix(y_true, y_pred, classes):
    
    n_classes = len(classes)
    
    matriz = np.zeros((n_classes, n_classes), dtype=int)
    
    mapa_indices = {label: i for i, label in enumerate(classes)}
    
    for real, predito in zip(y_true, y_pred):
    
        linha = mapa_indices[real]
        coluna = mapa_indices[predito]
        
        matriz[linha, coluna] += 1
        
    return matriz

In [48]:
def avaliacao(nome_modelo, modelo, X, y, test_size, n_rodadas=100):
    time_train = []
    time_test = []
    acuracias = []

    melhor_acc = -1
    pior_acc = float('inf')
    melhor_cm = None
    pior_cm = None

    classes = np.sort(np.unique(y))
    acc_p_classe = {c: [] for c in classes}

    for _ in tqdm(range(n_rodadas)):

        idx = np.random.permutation(X.shape[0])
        X, y = X[idx], y[idx]

        split = int(test_size * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Treinamento
        t0 = time()
        modelo.fit(X_train_scaled, y_train)
        time_train.append(time() - t0)

        # Predição
        t0 = time()
        y_pred = modelo.predict(X_test_scaled)
        time_test.append(time() - t0)

        acc = accuracy_score(y_test, y_pred)
        acuracias.append(acc)

        cm = confusion_matrix(y_test, y_pred, classes)

        diagonais = cm.diagonal()
        totais_linha = cm.sum(axis=1)
        for i, c in enumerate(classes):
            if totais_linha[i] > 0:
                acc_p_classe[c].append(diagonais[i] / totais_linha[i])

        if acc > melhor_acc:
            melhor_acc = acc
            melhor_cm = cm
        if acc < pior_acc:
            pior_acc = acc
            pior_cm = cm

        resultados = {
            'modelo': nome_modelo,
            'test_size': test_size,
            'tempo_treino_medio': np.mean(time_train),
            'tempo_teste_medio': np.mean(time_test),
            'acc_media': np.mean(acuracias),
            'acc_std': np.std(acuracias),
            'acc_max': np.max(acuracias),
            'acc_min': np.min(acuracias),
            'acc_mediana': np.median(acuracias),
            'melhor_cm': melhor_cm,
            'pior_cm': pior_cm,
            'acc_por_classe': {c: np.mean(acc_p_classe[c]) for c in classes}
        }
        return resultados 

In [16]:
def exibir_resultados(res):
    print(f"\n{'='*50}")
    print(f"Modelo: {res['modelo']} | Separação (Treino/Teste): {int((1-res['test_size'])*100)}/{int(res['test_size']*100)}")
    print(f"{'='*50}")
    print(f"Tempo Médio Treino: {res['tempo_treino_medio']:.6f}s | Tempo Médio Teste: {res['tempo_teste_medio']:.6f}s")
    print(f"Acurácia Média:   {res['acc_media']:.4f} ± {res['acc_std']:.4f}")
    print(f"Acurácia Máxima:  {res['acc_max']:.4f}")
    print(f"Acurácia Mínima:  {res['acc_min']:.4f}")
    print(f"Acurácia Mediana: {res['acc_mediana']:.4f}")
    print(f"\nAcurácia Média por Classe:")
    for c, acc in res['acc_por_classe'].items():
        print(f"  - {c}: {acc:.4f}")
    print(f"\nMelhor Matriz de Confusão:\n{res['melhor_cm']}")
    print(f"\nPior Matriz de Confusão:\n{res['pior_cm']}")

## **Questão 1 (Classificação de Padrões)** 

Objetivo: Abordar o clássico problema de classificação de patologias da coluna vertebral. O banco de dados está disponível no site abaixo:

https://archive.ics.uci.edu/dataset/212/vertebral+column     

Usando o problema com 3 classes, implementar e avaliar os classificadores estudados na disciplina, listados abaixo

Classificadores a Implementar
1. Classificador Vizinho Mais Próximo (distância de Minkowski de ordens $m \in {0,5; 2/3; 1; 3/2; 2; 5;2}$ )
2. Classificador Distância Mínima ao Centróide
3. Classificador Distância Mínima ao Centróide (versão robusta a outliers)
4. Classificador de Máxima Correlação

**OBS 1**: Computar os tempos de treinamento e teste de cada modelo para ajudar a decidir pelo melhor modelo.

**OBS 2**: Realizar, no mínimo, 100 rodadas de treinamento-teste independentes dos modelos e obter as estatísticas de desempenho (acurácia média, desvio-padrão, acurácia máxima, acurácia mínima e mediana da acurácia). Determinar a matriz de confusão para a melhor e pior rodada.

**OBS 3**:  Avaliar   o   efeito   da   porcentagem   de   separação   dos   dados   de   treino-teste   no   desempenho   dos classificadores. Sugestão: testar as seguintes possibilidades: 20/80, 30/70, 50/50, 70/30 e 80/20.

**OBS 4**: Calcular também as estatísticas de desempenho por classe, com o objetivo de entender se há classes mais fáceis de categorizar que outras. Boa sorte!

### Classificador Vizinho Mais Próximo (distância de Minkowski de ordens m ∈ {0,5; 2/3; 1; 3/2; 2; 5/2})

In [14]:
modelos_para_testar = []

In [28]:
proporcoes_teste = [0.8, 0.7, 0.5, 0.3, 0.2]

# Distâncias de Minkowski para o KNN: m in {0.5, 2/3, 1, 1.5, 2, 5}
m_values = [0.5, 2/3, 1.0, 1.5, 2.0, 5.0]

modelos_para_testar = []

In [ ]:
# 1. Adicionando as variações do KNN (k=1 conforme seu init padrão)
for m in m_values:
    modelos_para_testar.append((f"KNN (m={m:.2f})", KNN(k=1, m=m)))

In [9]:
sep = {'20/80': {'train': 0.2, 'test': 0.8},
       '30/70': {'train': 0.3, 'test': 0.7},
       '50/50': {'train': 0.5, 'test': 0.5},
       '70/30': {'train': 0.7, 'test': 0.3},
       '80/20': {'train': 0.8, 'test': 0.2}}

In [13]:
resultados_knn = {}

for k, v in sep.items():
    for m in [0.5, 2/3, 1, 3/2, 2, 5/2]:
        for _ in tqdm(range(30), desc=f"Rodada sep: {s}, m: {m:.2f}"):

            idx = np.random.permutation(X.shape[0])
            X, y = X[idx], y[idx]
            split = int(v['train'] * len(X))
            X_train, X_test = X[:split], X[split:]
            y_train, y_test = y[:split], y[split:]

            scaler = StandardScaler()
            scaler.fit(X_train)
            X_train_scaled = scaler.transform(X_train)

            model = KNN(k=1, m=m)
            model.fit(X=X_train_scaled, y=y_train)

            scaler_test = StandardScaler()
            scaler_test.fit(X_test)
            X_test_scaled = scaler_test.transform(X_test)

            y_pred = model.predict(X_test_scaled)            

Rodada sep: 80/20, m: 2.50: 100%|██████████| 30/30 [00:01<00:00, 22.70it/s]


### Classificador Distância Mínima ao Centróide

In [29]:
modelos_para_testar.append(("DMC", DMC()))

In [43]:
model = DMC()
model.fit(X, y)

y_pred = model.predict(X)

In [51]:
for proporcao in proporcoes_teste:
    print(f"\n\n{'#'*60}")
    print(f"AVALIANDO COM SEPARAÇÃO TREINO {int((1-proporcao)*100)}% / TESTE {int(proporcao*100)}%")
    print(f"{'#'*60}")
    
    for nome, modelo in modelos_para_testar:
        resultados = avaliacao(nome_modelo=nome, modelo=modelo, X=X, y=y, test_size=proporcao, n_rodadas=100)
        exibir_resultados(resultados)



############################################################
AVALIANDO COM SEPARAÇÃO TREINO 19% / TESTE 80%
############################################################


  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: unhashable type: 'numpy.ndarray'

In [33]:
resultados_mdc = {}

for k, v in sep.items():
    for _ in tqdm(range(100), desc=f"Rodada sep: {k}"):

        idx = np.random.permutation(X.shape[0])
        X, y = X[idx], y[idx]

        split = int(v['train'] * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)

        model = DMC()
        model.fit(X=X_train_scaled, y=y_train)

        scaler_test = StandardScaler()
        scaler_test.fit(X_test)
        X_test_scaled = scaler_test.transform(X_test)

        y_pred = model.predict(X_test_scaled)            

Rodada sep: 80/20: 100%|██████████| 100/100 [00:00<00:00, 1402.01it/s]


### Classificador Distância Mínima ao Centróide (versão robusta a outliers)

In [35]:
resultados_mdcr = {}

for k, v in sep.items():
    for _ in tqdm(range(100), desc=f"Rodada sep: {k}"):

        idx = np.random.permutation(X.shape[0])
        X, y = X[idx], y[idx]

        split = int(v['train'] * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)

        model = DMCR()
        model.fit(X=X_train_scaled, y=y_train)

        scaler_test = StandardScaler()
        scaler_test.fit(X_test)
        X_test_scaled = scaler_test.transform(X_test)

        y_pred = model.predict(X_test_scaled)            

Rodada sep: 80/20: 100%|██████████| 100/100 [00:00<00:00, 1458.31it/s]


### Classificador de Máxima Correlação

In [36]:
resultados_mdcr = {}

for k, v in sep.items():
    for _ in tqdm(range(100), desc=f"Rodada sep: {k}"):

        idx = np.random.permutation(X.shape[0])
        X, y = X[idx], y[idx]

        split = int(v['train'] * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)

        model = MaxCorr()
        model.fit(X=X_train_scaled, y=y_train)

        scaler_test = StandardScaler()
        scaler_test.fit(X_test)
        X_test_scaled = scaler_test.transform(X_test)

        y_pred = model.predict(X_test_scaled)            

Rodada sep: 80/20: 100%|██████████| 100/100 [00:00<00:00, 3034.34it/s]
